In [1]:
# V5 changes - adding new filtering logic for static images; separating lower
    # and upper face crops to filter out mid speech images
    # section #1 - added static and stability vars
    # section #2 - added filter_by_emotion_stability
    # section #3 - added static image filtering logic
    # section #4 - simplified analytics and print statements

In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from transformers import AutoImageProcessor, AutoModelForImageClassification
import cv2
import face_recognition
from PIL import Image
import os
import glob
from datetime import datetime
from tqdm import tqdm
import numpy as np
import pandas as pd
import shutil

In [3]:
# ==============================================================================
# 1. CONFIGURATION
# ==============================================================================
ANALYSIS_OUTPUT_ROOT = "/Users/natalyagrokh/AI/ml_expressions/img_expressions/flywheel"
MODEL_PATH = "/Users/natalyagrokh/AI/ml_expressions/img_expressions/sup_training/V29_20250710_082807"
CENTROIDS_PATH = "/Users/natalyagrokh/AI/ml_expressions/img_expressions/flywheel/emotion_centroids.pt"
RELEVANCE_THRESHOLD = 0.6 
STATIC_FRAME_THRESHOLD = 60 # Number of consecutive frames to identify a static object (e.g., 2 seconds at 30fps)
STABILITY_WINDOW = 5 # Rolling window size for the stability filter
STABILITY_THRESHOLD = 3 # How many times 

In [4]:
# ==============================================================================
# 2. UTILITY FUNCTIONS
# ==============================================================================

# Dynamically determines the next version number.
def get_next_version(base_dir):
    all_entries = glob.glob(os.path.join(base_dir, "V*_*"))
    existing = [os.path.basename(d) for d in all_entries if os.path.isdir(d)]
    versions = [int(d[1:].split("_")[0]) for d in existing if d.startswith("V") and "_" in d and d[1:].split("_")[0].isdigit()]
    return f"V{max(versions, default=0) + 1}"


# Runs the emotion recognition model on a single face image and returns a
# structured dictionary of probabilities.
def get_emotion_predictions(face_image, model, processor, device):
    # Use the processor to prepare the image for the model
    inputs = processor(images=face_image, return_tensors="pt").to(device)

    # Run inference
    with torch.no_grad():
        logits = model(**inputs).logits

    # Apply softmax to convert logits to probabilities
    probabilities = F.softmax(logits, dim=1).squeeze()

    # Get the top prediction
    top_confidence, top_pred_idx = torch.max(probabilities, dim=0)
    top_pred_label = model.config.id2label[top_pred_idx.item()]
    
    # Calculate entropy
    entropy = -torch.sum(probabilities * torch.log(probabilities + 1e-9)).item()
    
    # Create a dictionary with all results
    results = {
        "predicted_label": top_pred_label,
        "confidence": top_confidence.item(),
        "entropy": entropy
    }
    
    # This loop ensures all individual probabilities are added to the log for detailed analysis.
    for i, prob in enumerate(probabilities):
        label = model.config.id2label[i]
        results[f"prob_{label}"] = prob.item()
        
    return results
    

# Post-processes the log to filter for stable emotional states.
def filter_by_emotion_stability(df, window, threshold):
    if df.empty:
        return df
        
    print(f"\n--- Applying Emotion Stability Filter (Window={window}, Threshold={threshold}) ---")
    
    # This new list will store the indices of the rows we want to keep.
    stable_indices = []
    
    # Ensure the dataframe is sorted correctly before processing
    df = df.sort_values(by=['person_id', 'timestamp_seconds']).reset_index(drop=True)
    
    # Group by each unique person and iterate through their data
    for person_id, group in df.groupby('person_id'):
        labels = group['predicted_label']
        
        # Manually iterate through each prediction for this person
        for i in range(len(labels)):
            # Define the window of labels to check for stability
            current_window = labels.iloc[max(0, i - window + 1) : i + 1]
            
            # The emotion we are checking for stability is the most recent one
            current_label_to_check = labels.iloc[i]
            
            # Count how many times this emotion appears in the window
            if (current_window == current_label_to_check).sum() >= threshold:
                # If it's stable enough, keep it
                stable_indices.append(group.index[i])

    stable_df = df.loc[stable_indices].copy()
    
    print(f"-> Filtered {len(df) - len(stable_df)} unstable/transitional frames.")
    print(f"✅ {len(stable_df)} stable emotional events remain.")
    return stable_df

In [5]:
# ==============================================================================
# 3. CORE PROCESSING FUNCTION (with Face Identification & All Filters)
# ==============================================================================

# Processes video with all filters, ensuring file paths and person IDs are correctly logged.
def analyze_video_with_filters(video_path, save_dir, model, embedding_model, processor, device, centroids, relevance_threshold, static_threshold, process_every_n_frames=1):
    if not os.path.exists(video_path):
        print(f"❌ Error: Video file not found at {video_path}")
        return []

    video_capture = cv2.VideoCapture(video_path)
    total_frames = int(video_capture.get(cv2.CAP_PROP_FRAME_COUNT))
    fps = video_capture.get(cv2.CAP_PROP_FPS) if video_capture.get(cv2.CAP_PROP_FPS) > 0 else 30
    print(f"✅ Opened video: {os.path.basename(video_path)} ({total_frames} frames at {fps:.2f} fps)")

    # Create the directory for all face crops at the start
    face_crop_dir = os.path.join(save_dir, "face_crops")
    os.makedirs(face_crop_dir, exist_ok=True)

    # Data structures for tracking
    static_object_tracker = {}
    ignored_locations = set()
    known_face_encodings = []
    known_face_ids = []
    next_person_id = 1
    
    all_results_log = []
    
    pbar = tqdm(total=total_frames, desc="Analyzing Video")
    
    for frame_count in range(total_frames):
        ret, frame = video_capture.read()
        if not ret: break

        if frame_count % process_every_n_frames == 0:
            rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            face_locations = face_recognition.face_locations(rgb_frame)
            current_face_encodings = face_recognition.face_encodings(rgb_frame, face_locations)

            current_frame_locations = set()
            if current_face_encodings:
                for i, face_encoding in enumerate(current_face_encodings):
                    loc_key = face_locations[i]
                    current_frame_locations.add(loc_key)
                    
                    if loc_key in ignored_locations:
                        continue 

                    # Static Object Filter
                    if loc_key not in static_object_tracker:
                        static_object_tracker[loc_key] = {"count": 1, "last_frame": frame_count}
                    else:
                        if frame_count == static_object_tracker[loc_key]["last_frame"] + process_every_n_frames:
                            static_object_tracker[loc_key]["count"] += 1
                        else:
                            static_object_tracker[loc_key]["count"] = 1
                        static_object_tracker[loc_key]["last_frame"] = frame_count

                    if static_object_tracker[loc_key]["count"] > static_threshold:
                        if loc_key not in ignored_locations:
                            print(f"\n⚠️ Detected and ignoring static object at {loc_key}")
                            ignored_locations.add(loc_key)
                        continue

                    # Face Identification Logic
                    matches = face_recognition.compare_faces(known_face_encodings, face_encoding)
                    person_id = "Unknown"
                    if True in matches:
                        first_match_index = matches.index(True)
                        person_id = known_face_ids[first_match_index]
                    else:
                        person_id = f"Person_{next_person_id}"
                        known_face_encodings.append(face_encoding)
                        known_face_ids.append(person_id)
                        next_person_id += 1
                        print(f"\n✨ Discovered new face: {person_id}")

                    # Relevance Gate and Emotion Prediction
                    face_image = Image.fromarray(rgb_frame[loc_key[0]:loc_key[2], loc_key[3]:loc_key[1]])
                    inputs = processor(images=face_image, return_tensors="pt").to(device)
                    
                    with torch.no_grad():
                        embedding = embedding_model(**inputs).logits.squeeze()
                    
                    distances = {label_id: F.cosine_similarity(embedding, centroid, dim=0).item() for label_id, centroid in centroids.items()}
                    max_similarity = max(distances.values())
                    
                    if max_similarity >= relevance_threshold:
                        with torch.no_grad():
                            logits = model(**inputs).logits
                        emotion_results = get_emotion_predictions(face_image, model, processor, device)
                        
                        # 1. Create the full, absolute path for the cropped face.
                        face_filename = os.path.join(face_crop_dir, f"frame_{frame_count}_{person_id}_face_{i}.png")
                        
                        # 2. Save the image to that path.
                        face_image.save(face_filename)

                        # 3. Log this exact, verified path to the results.
                        log_entry = {
                            "timestamp_seconds": frame_count / fps,
                            "frame_number": frame_count,
                            "person_id": person_id,
                            "face_index": i,
                            "face_crop_path": face_filename,
                            "is_relevant": True,
                            "max_similarity": max_similarity,
                            **emotion_results
                        }
                        all_results_log.append(log_entry)

            # Clean up tracker for objects that are no longer detected
            stale_keys = [k for k, v in static_object_tracker.items() if k not in current_frame_locations]
            for k in stale_keys:
                del static_object_tracker[k]
                
        pbar.update(1)
        
    pbar.close()
    video_capture.release()
    
    print(f"✅ Video processing complete. Logged {len(all_results_log)} relevant emotional events from {len(known_face_ids)} unique person(s).")
    return all_results_log

In [6]:
# ==============================================================================
# 4. MAIN EXECUTION BLOCK
# ==============================================================================

# --- Setup Dynamic Save Directory ---
VERSION = get_next_version(ANALYSIS_OUTPUT_ROOT)
timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
VERSION_TAG = f"{VERSION}_{timestamp}"
SAVE_DIR = os.path.join(ANALYSIS_OUTPUT_ROOT, VERSION_TAG)
os.makedirs(SAVE_DIR, exist_ok=True)

print(f"📁 Created analysis output directory: {SAVE_DIR}")

# --- Load Models, Processor, and Centroids ---
print(f"\n--- Loading assets ---")
device = torch.device("mps" if torch.backends.mps.is_available() else "cpu")

# Load the main model for classification
model = AutoModelForImageClassification.from_pretrained(MODEL_PATH).to(device).eval()

# Create a second model instance for generating embeddings
embedding_model = AutoModelForImageClassification.from_pretrained(MODEL_PATH)
embedding_model.classifier = nn.Identity()
embedding_model.to(device).eval()

# Load the processor
processor = AutoImageProcessor.from_pretrained(MODEL_PATH)

# Load the pre-calculated emotion centroids
centroids = torch.load(CENTROIDS_PATH, map_location=device)
print(f"✅ Assets loaded onto {device}.")


# --- Run the Analysis ---
video_to_process = "/Users/natalyagrokh/AI/ml_expressions/img_expressions/flywheel/sample_vids/StreetQs.mp4" 

# Call the function with all required keyword arguments for clarity and safety
analysis_log = analyze_video_with_filters(
    video_path=video_to_process, 
    save_dir=SAVE_DIR, # <-- This was the missing argument
    model=model,
    embedding_model=embedding_model,
    processor=processor,
    device=device,
    centroids=centroids,
    relevance_threshold=RELEVANCE_THRESHOLD,
    static_threshold=STATIC_FRAME_THRESHOLD,
    process_every_n_frames=1
)

# --- Save Logs and Apply Stability Filter ---
if analysis_log:
    log_df = pd.DataFrame(analysis_log)
    full_log_path = os.path.join(SAVE_DIR, "emotion_log_before_stability_filter.csv")
    log_df.to_csv(full_log_path, index=False)
    print(f"\n✅ Full analysis log saved. Found {len(log_df)} relevant emotional events.")
    
    # Apply the final stability filter
    stable_log_df = filter_by_emotion_stability(log_df, window=STABILITY_WINDOW, threshold=STABILITY_THRESHOLD)
    
    # Save the final, clean log
    stable_log_path = os.path.join(SAVE_DIR, "final_stable_emotion_log.csv")
    stable_log_df.to_csv(stable_log_path, index=False)
    print(f"✅ Saved final stable emotion log to: {stable_log_path}")
else:
    print("\n⚠️ No relevant emotional events were detected.")

📁 Created analysis output directory: /Users/natalyagrokh/AI/ml_expressions/img_expressions/flywheel/V5_20250715_141530

--- Loading assets ---
✅ Assets loaded onto mps.
✅ Opened video: StreetQs.mp4 (5657 frames at 30.00 fps)


Analyzing Video:   0%|                         | 1/5657 [00:00<37:23,  2.52it/s]


✨ Discovered new face: Person_1


Analyzing Video:   1%|▎                       | 61/5657 [00:17<24:30,  3.81it/s]


⚠️ Detected and ignoring static object at (201, 634, 387, 448)


...


✨ Discovered new face: Person_20


Analyzing Video:  90%|███████████████████▉  | 5116/5657 [30:24<02:06,  4.26it/s]


✨ Discovered new face: Person_21


Analyzing Video: 100%|██████████████████████| 5657/5657 [32:23<00:00,  2.91it/s]


✅ Video processing complete. Logged 4122 relevant emotional events from 21 unique person(s).

✅ Full analysis log saved. Found 4122 relevant emotional events.

--- Applying Emotion Stability Filter (Window=5, Threshold=3) ---
-> Filtered 1089 unstable/transitional frames.
✅ 3033 stable emotional events remain.
✅ Saved final stable emotion log to: /Users/natalyagrokh/AI/ml_expressions/img_expressions/flywheel/V5_20250715_141530/final_stable_emotion_log.csv
